In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

In [3]:
train=pd.read_csv('../Input/train.csv')
test=pd.read_csv('../Input/test.csv')

In [8]:
new_condition_non_zero_odometer=train[(train['condition']=="new") & (train['odometer']>0)]

new_condition_non_zero_odometer_count = len(new_condition_non_zero_odometer)
new_condition_non_zero_odometer_count

54

In [10]:
# Drop the rows where condition is 'new' and odometer is greater than 0
train = train.drop(new_condition_non_zero_odometer.index)

In [13]:
df=train[train['fuel']=='electric']
df

,id,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,price
1726,1726,los angeles,2014,chevrolet,excellent,other,electric,57137,clean,automatic,fwd,compact,hatchback,black,NaN,2503
5912,5912,fredericksburg,1928,nissan,excellent,other,electric,40106,lien,automatic,fwd,compact,hatchback,black,va,8665
5978,5978,poconos,2014,nissan,good,4 cylinders,electric,23916,rebuilt,other,fwd,compact,hatchback,white,pa,6944
8594,8594,lexington,2014,nissan,like new,4 cylinders,electric,76980,rebuilt,automatic,fwd,full-size,sedan,black,ky,19001
10753,10753,eugene,2010,chevrolet,good,4 cylinders,electric,72748,clean,automatic,fwd,compact,hatchback,black,or,4440
12777,12777,mcallen / edinburg,2018,chevrolet,excellent,4 cylinders,electric,9951,rebuilt,automatic,fwd,compact,sedan,blue,NaN,36415
15498,15498,"washington, DC",1928,ford,like new,4 cylinders,electric,74421,clean,automatic,fwd,mid-size,sedan,blue,dc,32261
15722,15722,sarasota-bradenton,2016,honda,excellent,4 cylinders,electric,29757,salvage,manual,fwd,mid-size,sedan,blue,fl,36516
16752,16752,chico,2010,dodge,excellent,6 cylinders,electric,14620,clean,automatic,4wd,full-size,SUV,black,ca,21362
20195,20195,jacksonville,2015,honda,excellent,4 cylinders,electric,82561,salvage,automatic,fwd,mid-size,sedan,black,fl,1840


In [ ]:
train['price'] = np.log(train['price'])

In [ ]:
def preprocess(df):
    # 2999年以上のものは-1000するように
    df.loc[df['year'] >= 2999, 'year'] = df.loc[df['year'] >= 2999, 'year'] - 1000
    #cylinders列を数値型に
    df['cylinders'] = df['cylinders'].str.extract('(\d+)').astype(float)
    df['cylinders'] = df['cylinders'].replace('nan', 2.0)
    df['cylinders']=df['cylinders'].astype(float)
    
    #年間走行距離
    df['odometer_per_year'] = df['odometer'] / ((df['year'].max() + 1) - df['year'])
    #クレンジング
    df['manufacturer_cleaned'] = df['manufacturer'].apply(clean_manufacturer)
    #車の経過年数
    df['car_age'] = (df['year'].max() + 1) - df['year']
    #paint_colorをワンホットエンコーディング
    df = pd.get_dummies(df, columns=['paint_color'])
    #fuel列の欠損値をgasで補完
    df['fuel'] = df['fuel'].fillna('gas')
    #fuel列をラベルエンコーディング
    df['fuel'] = label_encoder.fit_transform(df['fuel'])
    df.fillna('nan', inplace=True)
    
    for c in ['region','manufacturer_cleaned','condition','fuel','title_status','transmission', 'drive', 'size', 'type', 'state']:
        df[c] = label_encoder.fit_transform(df[c].astype(str))
    #不要な列を削除
    del df['manufacturer']
    del df['type']
    return df